<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import psycopg2
import psycopg2.extras
import datetime
from datetime import timedelta, date

In [2]:
##connection to database

#setting credentials  -> only change credentals and execute
endpoint = "dbprojecteq.cefqf86eooid.us-east-1.rds.amazonaws.com"
dbname = "dbeq"
dbuser = "iaschwen"
password = "12345678"

#establish the connection by defining the cursor and autocommit
conn = psycopg2.connect("host="+endpoint+" dbname="+dbname+" user="+dbuser+" password="+password)
cur = conn.cursor()
conn.set_session(autocommit=True)


In [3]:
#dropping table before running it again
cur.execute('DROP TABLE datawarehouse_earthquake_tweetcount')

#creation of a DB-table
cur.execute("""CREATE TABLE IF NOT EXISTS datawarehouse_earthquake_tweetcount (
         id varchar(255),
         magnitude real,
         time varchar(255),
         type varchar(255),
         title varchar(255),
         coord_lat real,
         coord_long real,
         date varchar(255),
         count real
);
""")

In [4]:
def get_records():
    # SQL to get records from Postgres
    s = "SELECT * FROM quakes"
    # Error trapping
    try:
        # Execute the SQL
        cur.execute(s)
        # Retrieve records from Postgres into a Python List
        equakes = cur.fetchall()
    except psycopg2.Error as e:
        t_message = "Database error: " + e + "/n SQL: " + s
        return render_template("error.html", t_message = t_message)

    # Loop through the resulting list and print each user name, along with a line break:
    earthquakes_df = pd.DataFrame(equakes)
    return earthquakes_df

earthquakes_df = get_records()

In [5]:
earthquakes_df.columns=["id", "magnitude", "time", "type", "title", "coord_lat", "coord_long", "coord_height"]
earthquakes_df

,id,magnitude,time,type,title,coord_lat,coord_long,coord_height
0,usp000hsb7,5.0,1293752823930,earthquake,M 5.0 - Kermadec Islands region,-178.1350,-31.8300,None
1,usp000hsb2,5.0,1293744150350,earthquake,"M 5.0 - 108 km WSW of Isangel, Vanuatu",168.3530,-19.9840,None
2,usp000hsax,5.0,1293738996380,earthquake,"M 5.0 - 156 km WSW of Severo-Kuril’sk, Russia",153.9640,50.3800,None
3,ak010gqb24bq,5.0,1293736886463,earthquake,"M 5.0 - 113 km SE of Ugashik, Alaska",-156.1674,56.7391,None
4,usp000hsah,5.0,1293728174930,earthquake,"M 5.0 - 21 km NE of Pilar, Philippines",126.2490,9.9890,None
...,...,...,...,...,...,...,...,...
21817,us6000d3xn,5.7,1609225829079,earthquake,"M 5.7 - 212 km W of Bandon, Oregon",-126.9951,43.3890,None
21818,us6000d3x8,5.4,1609223009769,earthquake,"M 5.4 - 92 km W of Panguna, Papua New Guinea",154.6521,-6.3985,None
21819,us6000d3xi,5.0,1609222744705,earthquake,"M 5.0 - 92 km W of Panguna, Papua New Guinea",154.6586,-6.4549,None
21820,us6000d3x5,5.1,1609222417216,earthquake,"M 5.1 - 90 km WSW of Panguna, Papua New Guinea",154.6909,-6.4950,None


In [6]:
#create new column and change to date typ
earthquakes_df["date"] = earthquakes_df["time"]
# Change datatype from miliseconds to datetime (column "time") and from miliseconds to date (column "Date")
count = 0
for i in earthquakes_df.iloc[:, 8]:
    earthquakes_df.iloc[count, 8] = datetime.datetime.fromtimestamp(earthquakes_df.iloc[count, 8] / 1000.0)
    count += 1
earthquakes_df["time"] = pd.to_datetime(earthquakes_df['date'])
earthquakes_df["date"] = pd.to_datetime(earthquakes_df['date']).dt.date

In [7]:
earthquakes_df.head(2)

,id,magnitude,time,type,title,coord_lat,coord_long,coord_height,date
0,usp000hsb7,5.0,2010-12-31 00:47:03.930,earthquake,M 5.0 - Kermadec Islands region,-178.135,-31.830,None,2010-12-31
1,usp000hsb2,5.0,2010-12-30 22:22:30.350,earthquake,"M 5.0 - 108 km WSW of Isangel, Vanuatu",168.353,-19.984,None,2010-12-30


# tweetcount

In [8]:
def get_records_tweetcount():
    # SQL to get records from Postgres
    s = "SELECT * FROM tweetcount"
    # Error trapping
    try:
        # Execute the SQL
        cur.execute(s)
        # Retrieve records from Postgres into a Python List
        tweetcount = cur.fetchall()
    except psycopg2.Error as e:
        t_message = "Database error: " + e + "/n SQL: " + s
        return render_template("error.html", t_message = t_message)

    # Loop through the resulting list and print each user name, along with a line break:
    twitter_df = pd.DataFrame(tweetcount)
    return twitter_df

twitter_df = get_records_tweetcount()

In [9]:
twitter_df.columns=["starttime", "endtime", "count"]
twitter_df

,starttime,endtime,count
0,2021-09-30T00:00:00.000Z,2021-10-01T00:00:00.000Z,3037
1,2021-10-01T00:00:00.000Z,2021-10-02T00:00:00.000Z,2475
2,2021-10-02T00:00:00.000Z,2021-10-03T00:00:00.000Z,4769
3,2021-10-03T00:00:00.000Z,2021-10-04T00:00:00.000Z,2843
4,2021-10-04T00:00:00.000Z,2021-10-05T00:00:00.000Z,3609
...,...,...,...
3951,2011-01-15T00:00:00.000Z,2011-01-16T00:00:00.000Z,2422
3952,2011-01-16T00:00:00.000Z,2011-01-17T00:00:00.000Z,2302
3953,2011-01-17T00:00:00.000Z,2011-01-18T00:00:00.000Z,2489
3954,2011-01-18T00:00:00.000Z,2011-01-19T00:00:00.000Z,12291


In [10]:
twitter_df["starttime"] = pd.to_datetime(twitter_df["starttime"]).dt.date
# Rename column "starttime"
twitter_df.rename(columns={"starttime": "date"}, inplace=True)
# Drop Endtime
twitter_df.drop("endtime", axis=1, inplace=True)

In [12]:
earthquake_tweetcount = earthquakes_df.merge(twitter_df, on='date', how='left')

In [22]:
earthquake_tweetcount['count'] = earthquake_tweetcount['count'].fillna(0)
earthquake_tweetcount.drop("coord_height", axis=1, inplace=True)
earthquake_tweetcount[[1000]]

,id,magnitude,time,type,title,coord_lat,coord_long,date,count
0,usp000hsb7,5.0,2010-12-31 00:47:03.930,earthquake,M 5.0 - Kermadec Islands region,-178.1350,-31.8300,2010-12-31,0.0
1,usp000hsb2,5.0,2010-12-30 22:22:30.350,earthquake,"M 5.0 - 108 km WSW of Isangel, Vanuatu",168.3530,-19.9840,2010-12-30,0.0
2,usp000hsax,5.0,2010-12-30 20:56:36.380,earthquake,"M 5.0 - 156 km WSW of Severo-Kuril’sk, Russia",153.9640,50.3800,2010-12-30,0.0
3,ak010gqb24bq,5.0,2010-12-30 20:21:26.463,earthquake,"M 5.0 - 113 km SE of Ugashik, Alaska",-156.1674,56.7391,2010-12-30,0.0
4,usp000hsah,5.0,2010-12-30 17:56:14.930,earthquake,"M 5.0 - 21 km NE of Pilar, Philippines",126.2490,9.9890,2010-12-30,0.0
...,...,...,...,...,...,...,...,...,...
21817,us6000d3xn,5.7,2020-12-29 08:10:29.079,earthquake,"M 5.7 - 212 km W of Bandon, Oregon",-126.9951,43.3890,2020-12-29,15319.0
21818,us6000d3x8,5.4,2020-12-29 07:23:29.769,earthquake,"M 5.4 - 92 km W of Panguna, Papua New Guinea",154.6521,-6.3985,2020-12-29,15319.0
21819,us6000d3xi,5.0,2020-12-29 07:19:04.705,earthquake,"M 5.0 - 92 km W of Panguna, Papua New Guinea",154.6586,-6.4549,2020-12-29,15319.0
21820,us6000d3x5,5.1,2020-12-29 07:13:37.216,earthquake,"M 5.1 - 90 km WSW of Panguna, Papua New Guinea",154.6909,-6.4950,2020-12-29,15319.0


In [15]:
problemquakes = []
for index, row in earthquake_tweetcount.iterrows():
    try:
        cur.execute("""INSERT INTO datawarehouse_earthquake_tweetcount(id, magnitude, time, type, title, coord_lat, coord_long, date, count)
        VALUES ('{}',{},'{}','{}','{}',{},{},'{}',{});""".format(row['id'],row['magnitude'],row['time'],row['type'],row['title'],row['coord_lat'],row['coord_long'],row['date'],row['count']))
    except:
        problemquakes.append(row['id'])  # Save all the earthquakes that failed to fill into the DB in this list
        print("Problem with earthquake ('{}',{},'{}','{}','{}',{},{},'{}',{})".format(row['id'],row['magnitude'],row['time'],row['type'],row['title'],row['coord_lat'],row['coord_long'],row['date'],row['count']))